## Importar Librerías

In [1]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import os

#df = pd.read_csv('airline_passenger_satisfaction_cleaned.csv')
# Seleccionar las características (X) y la variable objetivo (y)
#X = df.drop(columns=['satisfaction'])  # Eliminar la columna objetivo del conjunto de características
#y = df['satisfaction']  # Columna objetivo

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## Cargar datos en formato CSV para ENTRENAMIENTO, VALIDACION y TEST 

### **Importancia de Cada Conjunto:**

- **Conjunto de Entrenamiento:** Utilizado para ajustar los pesos del modelo. Aquí, el modelo "aprende" de los datos.
- **Conjunto de Validación:** Utilizado para ajustar los hiperparámetros del modelo (como la profundidad de un árbol, la tasa de aprendizaje, el número de árboles, etc.) y para realizar la selección del modelo. Ayuda a determinar cuándo detener el entrenamiento para evitar sobreajuste.
- **Conjunto de Test:** Proporciona una evaluación final del rendimiento del modelo. Es esencial que este conjunto no se utilice para ajustar el modelo de ninguna manera para garantizar que el resultado refleje el rendimiento en datos no vistos.

In [2]:
# Crear la ruta si no existe
ruta = os.path.abspath('../data/modelos_entrenamiento/')

# Datos Entrenamiento
X_train = pd.read_csv(os.path.join(ruta, 'X_train.csv'))
y_train = pd.read_csv(os.path.join(ruta, 'y_train.csv'))

# Datos Validación
X_val = pd.read_csv(os.path.join(ruta, 'X_val.csv'))
y_val = pd.read_csv(os.path.join(ruta, 'y_val.csv'))

# Datos Test (Prueba)
X_test = pd.read_csv(os.path.join(ruta, 'X_test.csv'))
y_test = pd.read_csv(os.path.join(ruta, 'y_test.csv'))

# Verificar que las dimensiones de X e y coincidan para cada conjunto de datos
assert X_train.shape[0] == y_train.shape[0], "Error: El número de filas en X_train y y_train no coincide."
assert X_val.shape[0] == y_val.shape[0], "Error: El número de filas en X_val y y_val no coincide."
assert X_test.shape[0] == y_test.shape[0], "Error: El número de filas en X_test y y_test no coincide."

# Verificar que las columnas tengan nombres correctos y consistentes
assert 'satisfaction' in y_train.columns, "Error: El archivo y_train.csv debe tener una columna llamada 'satisfaction'."
assert 'satisfaction' in y_val.columns, "Error: El archivo y_val.csv debe tener una columna llamada 'satisfaction'."
assert 'satisfaction' in y_test.columns, "Error: El archivo y_test.csv debe tener una columna llamada 'satisfaction'."

print("Todos los archivos tienen un número de filas coincidente entre características y etiquetas, y la columna objetivo es 'satisfaction'.")

Todos los archivos tienen un número de filas coincidente entre características y etiquetas, y la columna objetivo es 'satisfaction'.


### Verificar si las variables son numericas para evitar error al entrenar el modelo

In [3]:
# Verificar tipos de datos de las etiquetas
print("Tipo de datos de y_train:", y_train.dtypes)
print("Tipo de datos de y_val:", y_val.dtypes)
print("Tipo de datos de y_test:", y_test.dtypes)

Tipo de datos de y_train: satisfaction    int64
dtype: object
Tipo de datos de y_val: satisfaction    int64
dtype: object
Tipo de datos de y_test: satisfaction    int64
dtype: object


## Separar características y etiquetas
Hay que separar las características (X) de la variable objetivo (y) para cada dataset.

In [4]:
# Separar las características (X) de la variable objetivo (y)
# Ya hemos cargado los archivos con solo características en X_train, X_val, X_test

# Separar la variable objetivo (y) - Asumimos que y_train, y_val, y_test solo tienen la columna 'satisfaction'
y_train = y_train['satisfaction']
y_val = y_val['satisfaction']
y_test = y_test['satisfaction']

print("Características y etiquetas separadas correctamente.")

Características y etiquetas separadas correctamente.


## Inicializar el modelo CatBoostClassifier

In [5]:
# Inicializar el modelo CatBoostClassifier - Erika
# Aquí se especifican los parámetros principales del modelo:
# - iterations: número de iteraciones (árboles) a entrenar
# - depth: profundidad de los árboles de decisión
# - learning_rate: tasa de aprendizaje para el ajuste de los pesos
# - verbose: cada cuántas iteraciones se imprime un resumen del progreso
model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, verbose=100)

# Entrenar el modelo utilizando el conjunto de entrenamiento y validación
# El parámetro eval_set se utiliza para especificar el conjunto de validación.
# early_stopping_rounds permite detener el entrenamiento si no hay mejora en el conjunto de validación
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

# Evaluar el modelo en el conjunto de prueba (test)
# El método score devuelve la exactitud (accuracy) del modelo en el conjunto de prueba
accuracy = model.score(X_test, y_test)
print(f"Exactitud del modelo en el conjunto de prueba: {accuracy:.2f}")

# Realizar predicciones en los conjuntos de entrenamiento y prueba
# predict() devuelve las etiquetas predichas para las muestras proporcionadas
catboost_train_preds = model.predict(X_train)
catboost_test_preds = model.predict(X_test)

# Calcular las probabilidades predichas para el conjunto de entrenamiento y prueba
# predict_proba() devuelve las probabilidades de clase; [:, 1] selecciona la probabilidad de la clase positiva
catboost_train_proba = model.predict_proba(X_train)[:, 1]
catboost_test_proba = model.predict_proba(X_test)[:, 1]

# Calcular las métricas de evaluación para los conjuntos de entrenamiento y prueba
# Recall (Sensibilidad): mide la capacidad del modelo para detectar todas las muestras positivas
# Usamos pos_label=1 porque las etiquetas son numéricas (0 para 'neutral or dissatisfied' y 1 para 'satisfied')
catboost_train_recall = recall_score(y_train, catboost_train_preds, pos_label=1)
catboost_test_recall = recall_score(y_test, catboost_test_preds, pos_label=1)

# F1 Score: media armónica de la precisión y la sensibilidad, útil en conjuntos de datos desbalanceados
catboost_train_f1 = f1_score(y_train, catboost_train_preds, pos_label=1)
catboost_test_f1 = f1_score(y_test, catboost_test_preds, pos_label=1)

# AUC (Area Under the Curve): mide la capacidad del modelo para distinguir entre clases
# No es necesario mapear las etiquetas ya que y_train y y_test son numéricas
catboost_train_auc = roc_auc_score(y_train, catboost_train_proba)
catboost_test_auc = roc_auc_score(y_test, catboost_test_proba)

# Imprimir las métricas de evaluación para analizar el rendimiento del modelo
print(f"CatBoost - Recall en Entrenamiento: {catboost_train_recall:.2f}")
print(f"CatBoost - Recall en Prueba: {catboost_test_recall:.2f}")
print(f"CatBoost - F1 Score en Entrenamiento: {catboost_train_f1:.2f}")
print(f"CatBoost - F1 Score en Prueba: {catboost_test_f1:.2f}")
print(f"CatBoost - AUC en Entrenamiento: {catboost_train_auc:.2f}")
print(f"CatBoost - AUC en Prueba: {catboost_test_auc:.2f}")


0:	learn: 0.5642258	test: 0.5643419	best: 0.5643419 (0)	total: 153ms	remaining: 1m 16s
100:	learn: 0.0954957	test: 0.1020326	best: 0.1020326 (100)	total: 1.38s	remaining: 5.43s
200:	learn: 0.0812890	test: 0.0933198	best: 0.0933198 (200)	total: 2.58s	remaining: 3.83s
300:	learn: 0.0732948	test: 0.0903986	best: 0.0903844 (299)	total: 3.85s	remaining: 2.55s
400:	learn: 0.0675110	test: 0.0891242	best: 0.0891138 (378)	total: 5.03s	remaining: 1.24s
499:	learn: 0.0624786	test: 0.0884543	best: 0.0884543 (499)	total: 6.44s	remaining: 0us

bestTest = 0.08845431341
bestIteration = 499

Exactitud del modelo en el conjunto de prueba: 0.96
CatBoost - Recall en Entrenamiento: 0.96
CatBoost - Recall en Prueba: 0.94
CatBoost - F1 Score en Entrenamiento: 0.97
CatBoost - F1 Score en Prueba: 0.96
CatBoost - AUC en Entrenamiento: 1.00
CatBoost - AUC en Prueba: 1.00


In [7]:
# Guarda el modelo CatBoost entrenado
model.save_model('../modelos/modelos_entrenamiento/catboost_model.cbm')
